## Import libraries

In [ ]:
import gc
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow_addons.optimizers import AdamW, Lookahead
from tensorflow.keras.layers import Activation, Input
from tensorflow.keras.layers import Embedding, Conv1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Concatenate, Add, LSTM
from tensorflow.keras.layers import Flatten, Dense, Dropout

## Prepare data for model training

In [ ]:
with open("../input/tps-june-data-preprocess/TPS_June_Dataset_Set1.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']
test_df = processed_data['test_df']

col_list = test_df.columns.to_list()

del processed_data
gc.collect()

In [ ]:
Xtrain = train_df.loc[:, train_df.columns != 'target'].copy()
Ytrain = train_df['target'].copy()
Ytrain_oh = pd.get_dummies(train_df['target']).copy()
Xtest = test_df.copy()

print("Xtrain: {} \nYtrain: {} \nYtrain_oh: {} \nXtest: {}".format(Xtrain.shape, Ytrain.shape, 
                                                                   Ytrain_oh.shape, Xtest.shape))

del train_df
del test_df
gc.collect()

## Define TPU config

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

In [ ]:
mini_batch_size = strategy.num_replicas_in_sync * 32
print(f'batch size: {mini_batch_size}')

## Build the model

In [ ]:
def dnn_model(n_features):
    
    x_input = Input(shape=(n_features,))
    
    x = Embedding(512, 16)(x_input)
    
    x = Conv1D(filters=32, kernel_size=3, 
               strides=2, padding='same', 
               kernel_regularizer=l2(0.0003),
               kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    
    x = Conv1D(filters=64, kernel_size=3, 
               strides=2, padding='same', 
               kernel_regularizer=l2(0.0003),
               kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    
    x = LSTM(units=96, activation='swish',
             return_sequences=False, dropout=0.1,
             kernel_regularizer=l2(0.0003),
             kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    
    x = Dense(units=16, kernel_initializer='he_uniform', 
                kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)
    x = Dropout(rate=0.1)(x)

    x_output = Dense(units=9, activation='softmax', 
                     kernel_initializer='he_uniform')(x)

    model = Model(inputs=x_input, outputs=x_output, 
                  name='DNN_Model')
    return model

In [ ]:
model = dnn_model(Xtrain.shape[1])
model.compile(loss='categorical_crossentropy',
              optimizer=Lookahead(AdamW(lr=1e-2, 
                                        weight_decay=1e-5, 
                                        clipvalue=700), 
                                  sync_period=10))
model.summary()

In [ ]:
FOLD = 10
NUM_SEED = 3
VERBOSE = 0

np.random.seed(3)
seeds = np.random.randint(0, 100, size=NUM_SEED)

counter = 0
oof_score = 0
y_pred_meta_dnn = np.zeros((Xtrain.shape[0], 9))
y_pred_final_dnn = np.zeros((Xtest.shape[0], 9))

In [ ]:
with strategy.scope():
    
    for sidx, seed in enumerate(seeds):
        seed_score = 0

        kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

        for idx, (train, val) in enumerate(kfold.split(Xtrain, Ytrain)):
            counter += 1

            train_x, train_y, train_y_oh = Xtrain.iloc[train], Ytrain.iloc[train], Ytrain_oh.iloc[train]
            val_x, val_y, val_y_oh = Xtrain.iloc[val], Ytrain.iloc[val], Ytrain_oh.iloc[val]

            model = dnn_model(Xtrain.shape[1])
            model.compile(loss='categorical_crossentropy',
                          optimizer=Lookahead(AdamW(lr=1e-2, 
                                                    weight_decay=1e-5, 
                                                    clipvalue=700), 
                                              sync_period=10))

            early = EarlyStopping(monitor="val_loss", mode="min", 
                                  restore_best_weights=True, 
                                  patience=7, verbose=VERBOSE)

            reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.25, 
                                          min_lr=1e-6, patience=3, 
                                          verbose=VERBOSE, mode='min')
            
            save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')

            chk_point = ModelCheckpoint('./DNN_model.h5', options=save_locally, 
                                        monitor='val_loss', verbose=VERBOSE, 
                                        save_best_only=True, mode='min')

            history = model.fit(
                train_x, train_y_oh, 
                batch_size=mini_batch_size,
                epochs=250, 
                verbose=VERBOSE, 
                callbacks=[reduce_lr, early, chk_point], 
                validation_data=(val_x, val_y_oh)
            )
            
            load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')

            model = load_model('./DNN_model.h5', options=load_locally)

            y_pred = model.predict(val_x)
            y_pred_meta_dnn[val] += y_pred
            y_pred_final_dnn += model.predict(Xtest)

            score = log_loss(val_y_oh, y_pred)
            oof_score += score
            seed_score += score
            print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))

        print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))

In [ ]:
y_pred_meta_dnn = y_pred_meta_dnn / float(NUM_SEED)
y_pred_final_dnn = y_pred_final_dnn / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

In [ ]:
np.savez_compressed('./DNN_Meta_Features.npz',
                    y_pred_meta_dnn=y_pred_meta_dnn, 
                    oof_score=oof_score,
                    y_pred_final_dnn=y_pred_final_dnn)

## Create submission file

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
submit_df = pd.DataFrame()
submit_df['id'] = test_df['id']
submit_df['Class_1'] = y_pred_final_dnn[:,1]
submit_df['Class_2'] = y_pred_final_dnn[:,2]
submit_df['Class_3'] = y_pred_final_dnn[:,3]
submit_df['Class_4'] = y_pred_final_dnn[:,4]
submit_df['Class_5'] = y_pred_final_dnn[:,5]
submit_df['Class_6'] = y_pred_final_dnn[:,6]
submit_df['Class_7'] = y_pred_final_dnn[:,7]
submit_df['Class_8'] = y_pred_final_dnn[:,8]
submit_df['Class_9'] = y_pred_final_dnn[:,0]
submit_df.head()

In [ ]:
submit_df.to_csv("./DNN_submission.csv", index=False)